In [1]:
from image import Image

Reading config file: /home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No validation file provided. Turning off validation loop


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [2]:
import pandas as pd
phenogeo_csv = "/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/temp_data/phenogeo.csv"
df = pd.read_csv(phenogeo_csv)
df = pd.read_csv(phenogeo_csv).dropna(subset=['adjNorthing', 'adjEasting', 'uid'])
df = df.drop_duplicates(subset='uid')
df.reset_index(drop=True, inplace=True)

sample_image_path = "/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/input_data/rgb_test/2021_GRSM_5_275000_3951000_image.tif"

img = Image(sample_image_path)
img.annotate(df)

for tree in img.generate_hsi_trees("/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/input_data/hsi/NEON_D07_GRSM_DP3_275000_3951000_reflectance.h5"): print(tree[1])

/tmp/ipykernel_310973/1992762650.py:3: DtypeWarning: Columns (0,1,2,3,4,7,11,13,21,22,24,25,27,28,29,31,32,33,34,35,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,64,68,70,78,79,80,81,82,83,84,85,86,88,89,90,91,93,94,99,107,111,112,113,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(phenogeo_csv)
/tmp/ipykernel_310973/1992762650.py:4: DtypeWarning: Columns (0,1,2,3,4,7,11,13,21,22,24,25,27,28,29,31,32,33,34,35,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,64,68,70,78,79,80,81,82,83,84,85,86,88,89,90,91,93,94,99,107,111,112,113,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(phenogeo_csv).dropna(subset=['adjNorthing', 'adjEasting', 'uid'])
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does no

Predicting DataLoader 0: 100%|██████████| 729/729 [00:14<00:00, 51.26it/s]
35618 predictions in overlapping windows, applying non-max supression
20543 predictions kept after non-max suppression


/home/kmitchell/miniconda3/envs/micropheno/lib/python3.11/site-packages/deepforest/utilities.py:476: UserWarning: This method is deprecated and will be removed in version DeepForest 2.0.0, please use boxes_to_shapefile which unifies project_boxes and annotations_to_shapefile functionalities
  warnings.warn("This method is deprecated and will be "


xmin                                   3461.0
ymin                                   7952.0
xmax                                   3500.0
ymax                                   7998.0
label                                    Tree
                                  ...        
otherTagOrg                               NaN
samplingProtocolVersion     NEON.DOC.000987vE
identificationReferences                  NaN
morphospeciesID                           NaN
morphospeciesIDRemarks                    NaN
Name: 0, Length: 134, dtype: object
xmin                                   3461.0
ymin                                   7952.0
xmax                                   3500.0
ymax                                   7998.0
label                                    Tree
                                  ...        
otherTagOrg                               NaN
samplingProtocolVersion     NEON.DOC.000987vE
identificationReferences                  NaN
morphospeciesID                           Na

In [3]:
img.gdf['labeled'].value_counts()[True]


7

In [4]:
df = img.gdf[img.gdf['labeled'] == True]

In [16]:
img.ground_truth_df['date']

0     2015-10-13
1     2015-10-13
2     2015-10-13
3     2015-10-13
4     2015-10-13
5     2015-10-13
6     2015-10-13
7     2015-10-13
8     2015-10-13
9     2015-10-13
10    2015-10-13
11    2015-10-13
12    2015-10-13
13    2015-10-13
14    2015-10-13
15    2015-10-13
16    2015-10-13
17    2015-10-13
18    2015-10-13
19    2015-10-13
20    2015-10-13
21    2015-10-13
22    2015-10-13
23    2015-10-13
24    2015-10-13
25    2015-10-13
26    2015-10-13
27    2015-10-13
28    2021-10-25
Name: date, dtype: object

In [6]:
for tree in hsi_trees:
    print(tree)

NameError: name 'hsi_trees' is not defined